### Start from here


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf


In [2]:

# Load the data
data = pd.read_csv('/content/drive/MyDrive/car_price_prediction.csv')

# drop rows where the price is less than 100
data = data[data['Price'] >= 100]
data = data[~data['Model'].isin(["GONOW","IVECO DAYLY"])]



# Drop the totalPrice column
data = data.drop('totalPrice', axis=1)
data = data.drop('Wheel', axis=1)
data = data.drop('no-tax', axis=1)
data = data.drop('Engine volume', axis=1)

data = data.drop("ID", axis=1)

data['Doors'] = data['Doors'].replace({'-May':'','-Mar':''}, regex=True)
data['Doors'] = data['Doors'].str.replace('>5','5')

# Drop rows with missing or empty values in the "Doors" column
data = data[data['Doors'].str.strip() !='']
data['Doors'] = data['Doors'].astype(int)


# Gear box editing
data['Gear box type'] = data['Gear box type'].replace({'Automatic': 1, 'Tiptronic': 2,'Variator':3, 'Manual':4})
data = data[data['Gear box type'].isin([1,2,3,4])]

data["Gear box type"] = data["Gear box type"].astype(int)

# 'Automatic' is changed to 1
# 'Tiptronic' is changed to 2
# 'Variator' is changed to 3
# 'Manual' is changed to 4


# Replace values in the "Leather interior" column based on conditions
data["Leather interior"] = data["Leather interior"].replace({"Yes": 1, "No": 0}) # Yes is 1; No is 0
# keep the rows where the value of "Leather interior" is 0 or 1
data = data[data["Leather interior"].apply(lambda x: x in [0, 1])]


# fuel type edit
data["Fuel type"] = data["Fuel type"].replace({"Hybrid":1, "Petrol":2,"Diesel":3,"CNG":4,"Plug-in Hybrid":5,"LPG":6,"Hydrogen":7})
allowed_fuel_types = [1, 2, 3, 4, 5, 6, 7]
data = data[data["Fuel type"].apply(lambda x: x in allowed_fuel_types)]

# 'Hybrid' is changed to 1,
# 'Petrol' is changed to 2, 
# 'Diesel' is changed to 3, 
# 'CNG' is changed to 4, 
# 'Plug-in Hybrid' is changed to 5, 
# 'LPG' is changed to 6,
# 'Hydrogen' is changed to 7


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/car_price_prediction.csv'

In [ ]:
# Mapping

data["Manufacturer"] = data["Manufacturer"].replace({"LEXUS":1, "CHEVROLET":2,"GREATWALL":3,"HONDA":4,"FORD":5,"HYUNDAI":6,"TOYOTA":7,
                                                     "MERCEDES-BENZ":8, "OPEL":9,"PORSCHE":10,"BMW":11,"JEEP":12,"VOLKSWAGEN":13,"AUDI":14,
                                                     "RENAULT":15, "NISSAN":16,"SUBARU":17,"DAEWOO":18,"KIA":19,"MITSUBISHI":20,"SSANGYONG":21,
                                                     "MAZDA":22, "GMC":23,"FIAT":24,"INFINITI":25,"ALFA ROMEO":26,"SUZUKI":27,"ACURA":28,
                                                     "LINCOLN":29, "VAZ":30,"GAZ":31,"CITROEN":32,"LAND ROVER":33,"MINI":34,"DODGE":35,
                                                     "CHRYSLER":36, "JAGUAR":37,"ISUZU":38,"SKODA":39,"DAIHATSU":40,"BUICK":41,"TESLA":42,
                                                     "CADILLAC":43, "PEUGEOT":44,"BENTLEY":45,"VOLVO":46,"HAVAL":47,"HUMMER":48,"SCION":49,
                                                     "UAZ":50, "MERCURY":51,"ZAZ":52,"ROVER":53,"SEAT":54,"LANCIA":55,"MOSKVICH":56,
                                                     "MASERATI":57, "FERRARI":58,"SAAB":59,"LAMBORGHINI":60,"PONTIAC":61,"SATURN":62,"ASTON MARTIN":63})


: 

In [ ]:
# Remove "KM"
data["Mileage"] = data["Mileage"].str.replace("km","")
data["Mileage"] = data["Mileage"].astype(float)

: 

In [ ]:
data["Category"] = data["Category"].replace({"Jeep":1, "Hatchback":2,"Sedan":3,"Microbus":4,"Goods wagon":5,"Universal":6,"Coupe":7,
                                             "Minivan":8, "Cabriolet":9,"Limousine":10,"Pickup":11})

: 

In [ ]:
# data["Color"] = data["Color"].replace({"Silver":1, "Black":2,"White":3,"Grey":4,"Blue":5,"Green":6,"Red":7,
#                                                      "Sky blue":8, "Orange":9,"Yellow":10,"Golden":11,"Beige":12,"Brown":13,"Carnelian red":14,
#                                                      "Purple":15, "Pink":16})

# data = pd.get_dummies(data, columns=["Color"])


data = data.drop('Color', axis=1)

: 

In [ ]:
data["Drive wheels"] = data["Drive wheels"].replace({"4x4":1, "Front":2,"Rear":3})

: 

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
encoder = LabelEncoder()

# Fit the encoder on the "Model" feature
encoder.fit(data["Model"])

# Transform the "Model" feature to numeric
data["Model"] = encoder.transform(data["Model"])


# Initialize the encoder
encoder1 = LabelEncoder()

# Fit the encoder on the "Model" feature
encoder1.fit(data["Manufacturer"])

# Transform the "Model" feature to numeric
data["Manufacturer"] = encoder1.transform(data["Manufacturer"])


: 

In [ ]:
# Cat_types = data["Manufacturer"].unique()
# p = 0
# for c in Cat_types:
#     p += 1  
#     print(p,c)

: 

In [ ]:
# # Assign each value of the "Model" column with the integer value of the corresponding "Manufacturer"
# data["Model_enc"] = data["Manufacturer"]


: 

In [ ]:
print(data.iloc[0])


: 

In [ ]:
data.columns


: 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


# Define the features and labels
X = data.drop("Price", axis=1)
y = data["Price"]

class CarPriceEstimator(nn.Module):
    def __init__(self):
        super(CarPriceEstimator, self).__init__()
        self.layer1 = nn.Linear(12, 60)
        self.layer2 = nn.Linear(60, 30)
        # self.layer3 = nn.Linear(150, 65)
        # self.layer4 = nn.Linear(65, 30)
        self.output_layer = nn.Linear(30, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        # x = torch.relu(self.layer3(x))
        # x = torch.tanh(self.layer4(x))
        x = self.output_layer(x)
        return x


# normalize data 
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.values.reshape(-1, 1))

# Split the data into training and test sets
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=42)


# Convert data to Pytorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()


# instantiate the model
model = CarPriceEstimator()

# define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)

# train the model
for epoch in range(1500):
    optimizer.zero_grad()
    # forward pass
    y_pred = model(X_train)
    # compute the loss
    loss = criterion(y_pred, y_train)
    # backward pass and optimize
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}/1500: Loss = {loss.item()}')

with torch.no_grad():
    y_val_pred = model(X_val)
    val_loss = criterion(y_val_pred, y_val)    

    print("validation loss:", val_loss.item())

    y_test_pred = model(X_test)
    test_loss = criterion(y_test_pred, y_test)

    print("test loss:", test_loss.item())

    y_pred = model(X_test)
    y_pred = y_pred.numpy()

    y_pred = scaler.inverse_transform(y_pred)

    print(y_pred)

model.eval()

with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.numpy()
    y_pred = scaler.inverse_transform(y_pred)
    print(y_pred)

torch.save(model.state_dict(), 'CarPriceEstimator.pt')

loaded_model = CarPriceEstimator()
loaded_model.load_state_dict(torch.load('CarPriceEstimator.pt'))
loaded_model.eval()

with torch.no_grad():
    y_pred = loaded_model(X_test)
    y_pred = y_pred.numpy()
    y_pred = scaler.inverse_transform(y_pred)
    print(y_pred)




: 

In [ ]:
# import pickle

# # Save the scaler
# with open('scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

# # Load the scaler
# with open('scaler.pkl', 'rb') as f:
#     scaler = pickle.load(f)

# test_data = X.iloc[1246]
# test_data = scaler.transform(np.array(test_data).reshape(1,-1))
# test_data = torch.from_numpy(test_data).float()



: 

In [ ]:
# import pickle
# import torch
# import numpy as np
# from sklearn.preprocessing import StandardScaler

# # Load the scaler
# with open('scaler.pkl', 'rb') as f:
#     scaler = pickle.load(f)

# # load the model
# loaded_model = CarPriceEstimator()
# loaded_model.load_state_dict(torch.load('CarPriceEstimator.pt'))
# loaded_model.eval()

# # Scale and normalize the test data
# test_data = X.iloc[1246]
# test_data = np.array(test_data).reshape(1, -1)
# test_data = scaler.transform(test_data)
# test_data = torch.from_numpy(test_data).float()

# # Make the prediction
# with torch.no_grad():
#     output = loaded_model(test_data)

# # Inverse-transform the output to get the final prediction in the original scale
# prediction = scaler.inverse_transform(output.numpy().reshape(-1, 1))
# print(prediction)


: 

In [ ]:
def predict(untrained_example):
    # Scale the untrained example
    example = scaler.transform(np.array(untrained_example).reshape(1, -1))
    # Convert to Pytorch tensor
    example = torch.from_numpy(example).float()
    # Pass through the trained model to make a prediction
    with torch.no_grad():
        output = model(example)
    # Inverse-transform the output to get the final prediction in the original scale
    prediction = scaler.inverse_transform(output.numpy())
    return prediction


: 

In [ ]:

# Make the prediction
loaded_model = CarPriceEstimator()
loaded_model.load_state_dict(torch.load('CarPriceEstimator.pt'))
loaded_model.eval()
output = loaded_model.predict(test_data)
print(output)


: 

In [ ]:
loaded_model = CarPriceEstimator()
loaded_model.load_state_dict(torch.load('CarPriceEstimator.pt'))
loaded_model.eval()

test_data = ... # some data you want to predict on
output = predict(test_data)
print(output)


: 

-------------------------

In [ ]:
model.eval()

: 

In [ ]:
# Print the car prices
with torch.no_grad():
    y_pred = model(X_test[:5])
    y_pred = y_pred.numpy()
    y_pred = scaler.inverse_transform(y_pred)
    print(y_pred)



: 

------------------------NOT WORKING------------------------

In [ ]:
from keras.optimizers.optimizer_v2 import gradient_descent
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

# Define the features and labels
X = data.drop("Price", axis=1)


y = data["Price"]

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# from sklearn.preprocessing import QuantileTransformer
# scaler = QuantileTransformer(output_distribution='normal')
# y = scaler.fit_transform(y.values.reshape(-1, 1))


# scaler = MinMaxScaler(feature_range=(0, 1))
# y = scaler.fit_transform(y.values.reshape(-1, 1))




# # Split the data into training and test sets
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=42)


# Convert data to Pytorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train.values).float().reshape(-1,1)
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val.values).float().reshape(-1,1)
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test.values).float().reshape(-1,1)


# ---------------------------------------------------------------------------------------------------------------
# # Normalizing 
# tf.keras.layers.Normalization(
#     axis=-1, mean=None, variance=None, invert=False)


# Define the model
model = Sequential([
    # keras.layers.Reshape(target_shape=(29 * 28,), input_shape=(28, 28)),
    Dense(units=12, activation='relu'),
    Dense(units=60, activation='relu'),
    Dense(units=120, activation='relu'),
    Dense(units=120, activation='relu'),
    Dense(units=60, activation='relu'),
    Dense(units=1, activation='linear')
])

# Compile the model
from keras.optimizers import Adagrad
optimizer = Adagrad(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# model.compile(optimizer='adam',loss='mean_squared_error')

# ---------------------------------------------------------------------------------------------------------------
import torch
import torch.nn as nn
import torch.optim as optim

class CarPriceEstimator(nn.Module):
    def __init__(self):
        super(CarPriceEstimator, self).__init__()
        self.layer1 = nn.Linear(12, 60)
        self.layer2 = nn.Linear(60, 120)
        self.layer3 = nn.Linear(120, 60)
        self.output_layer = nn.Linear(60, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = self.output_layer(x)
        return x


# instantiate the model
model = CarPriceEstimator()

# define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# train the model
for epoch in range(1000):
    optimizer.zero_grad()
    # forward pass
    y_pred = model(X_train)
    # compute the loss
    loss = criterion(y_pred, y_train)
    # backward pass and optimize
    loss.backward()
    optimizer.step()






: 

In [ ]:
# y.mean()
X.mean()
# y.std()


: 

In [ ]:
num_features = X_train
print(num_features)

: 

In [ ]:
model.build((12958, 12))
print(model.summary())

: 

In [ ]:
# Train the model on the training data
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

: 

: 

 -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# NOT WORKING

#------------------------------------------------------------------------------

# Split the data into features (inputs) and target (output)
X = data.drop('Price', axis=1)
y = data['Price'].astype('float32')


# Normalize the input features
X_num = X.select_dtypes(include=['float', 'int'])
X_num = (X_num - X_num.mean()) / X_num.std()

# Shuffle the data
X_num, y = shuffle(X_num, y, random_state=42)

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#Define the model
model = Sequential()

#Add layers to the model
model.add(Dense(32, input_dim=X_num.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(optimizer=Adam(learning_rate=0.001), loss=root_mean_squared_error)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
test_loss = test_loss.ravel()[0]
print('Test loss:', test_loss)

# Save the model
model.save('used_car_price_model.h5')

# Save the training history
import pickle
with open('used_car_price_history', 'wb') as f:
    pickle.dump(history.history, f)



: 

In [ ]:
# ALL THE CODE 

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
tf.keras.backend.set_floatx('float32')


# Load the data
data = pd.read_csv('/content/drive/MyDrive/car_price_prediction.csv')

# drop rows where the price is less than 100
data = data[data['Price'] >= 100]
data = data[~data['Model'].isin(["GONOW","IVECO DAYLY"])]



# Drop the totalPrice column
data = data.drop('totalPrice', axis=1)
data = data.drop('Wheel', axis=1)
data = data.drop('no-tax', axis=1)
data = data.drop('Engine volume', axis=1)
data['Doors'] = data['Doors'].replace({'-May':'','-Mar':''}, regex=True)
data['Doors'] = data['Doors'].str.replace('>5','5')

# Drop rows with missing or empty values in the "Doors" column
data = data[data['Doors'].str.strip() !='']
data['Doors'] = data['Doors'].astype(int)

data['Doors'] = data['Doors'].astype(int)

# Gear box editing
data['Gear box type'] = data['Gear box type'].replace({'Automatic': 1, 'Tiptronic': 2,'Variator':3, 'Manual':4})
data = data[data['Gear box type'].isin([1,2,3,4])]

data["Gear box type"] = data["Gear box type"].astype(int)

# 'Automatic' is changed to 1
# 'Tiptronic' is changed to 2
# 'Variator' is changed to 3
# 'Manual' is changed to 4


# Replace values in the "Leather interior" column based on conditions
data["Leather interior"] = data["Leather interior"].replace({"Yes": 1, "No": 0}) # Yes is 1; No is 0
# keep the rows where the value of "Leather interior" is 0 or 1
data = data[data["Leather interior"].apply(lambda x: x in [0, 1])]


# fuel type edit
data["Fuel type"] = data["Fuel type"].replace({"Hybrid":1, "Petrol":2,"Diesel":3,"CNG":4,"Plug-in Hybrid":5,"LPG":6,"Hydrogen":7})
allowed_fuel_types = [1, 2, 3, 4, 5, 6, 7]
data = data[data["Fuel type"].apply(lambda x: x in allowed_fuel_types)]

# 'Hybrid' is changed to 1,
# 'Petrol' is changed to 2, 
# 'Diesel' is changed to 3, 
# 'CNG' is changed to 4, 
# 'Plug-in Hybrid' is changed to 5, 
# 'LPG' is changed to 6,
# 'Hydrogen' is changed to 7

# Mapping

data["Manufacturer"] = data["Manufacturer"].replace({"LEXUS":1, "CHEVROLET":2,"GREATWALL":3,"HONDA":4,"FORD":5,"HYUNDAI":6,"TOYOTA":7,
                                                     "MERCEDES-BENZ":8, "OPEL":9,"PORSCHE":10,"BMW":11,"JEEP":12,"VOLKSWAGEN":13,"AUDI":14,
                                                     "RENAULT":15, "NISSAN":16,"SUBARU":17,"DAEWOO":18,"KIA":19,"MITSUBISHI":20,"SSANGYONG":21,
                                                     "MAZDA":22, "GMC":23,"FIAT":24,"INFINITI":25,"ALFA ROMEO":26,"SUZUKI":27,"ACURA":28,
                                                     "LINCOLN":29, "VAZ":30,"GAZ":31,"CITROEN":32,"LAND ROVER":33,"MINI":34,"DODGE":35,
                                                     "CHRYSLER":36, "JAGUAR":37,"ISUZU":38,"SKODA":39,"DAIHATSU":40,"BUICK":41,"TESLA":42,
                                                     "CADILLAC":43, "PEUGEOT":44,"BENTLEY":45,"VOLVO":46,"HAVAL":47,"HUMMER":48,"SCION":49,
                                                     "UAZ":50, "MERCURY":51,"ZAZ":52,"ROVER":53,"SEAT":54,"LANCIA":55,"MOSKVICH":56,
                                                     "MASERATI":57, "FERRARI":58,"SAAB":59,"LAMBORGHINI":60,"PONTIAC":61,"SATURN":62,"ASTON MARTIN":63})

# Remove "KM"
data["Mileage"] = data["Mileage"].str.replace("km","")
data["Mileage"] = data["Mileage"].astype(float)

data["Category"] = data["Category"].replace({"Jeep":1, "Hatchback":2,"Sedan":3,"Microbus":4,"Goods wagon":5,"Universal":6,"Coupe":7,
                                             "Minivan":8, "Cabriolet":9,"Limousine":10,"Pickup":11})

data["Color"] = data["Color"].replace({"Silver":1, "Black":2,"White":3,"Grey":4,"Blue":5,"Green":6,"Red":7,
                                                     "Sky blue":8, "Orange":9,"Yellow":10,"Golden":11,"Beige":12,"Brown":13,"Carnelian red":14,
                                                     "Purple":15, "Pink":16})

data = pd.get_dummies(data, columns=["Color"])

data["Drive wheels"] = data["Drive wheels"].replace({"4x4":1, "Front":2,"Rear":3})

from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
encoder = LabelEncoder()

# Fit the encoder on the "Model" feature
encoder.fit(data["Model"])

# Transform the "Model" feature to numeric
data["Model"] = encoder.transform(data["Model"])


: 